# Full Chromatin Simulation with Adaptable Refinement

In this notebook, we demonstrate a Monte Carlo simulation of chromatin organization where the level of detail is adaptibly confined to improve convergence onto a globally optimal configuration.

### Import Modules

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys
from inspect import getmembers, isfunction

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)
os.chdir(parent_dir)

print("Root Directory: ")
print(os.getcwd())

Root Directory: 
/home/users/jwakim/CodeDev/chromo_adaptable_refinement/chromo


In [5]:
import chromo.mc as mc
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import UniformDensityField
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name
from chromo.util.poly_paths import gaussian_walk
import chromo.util.rediscretize as rd
import doc.tools.mu_schedules as ms

### Generate Initial Chromatin Fiber

In [6]:
# Binders
hp1 = chromo.binders.get_by_name("HP1")
hp1.chemical_potential = -0.4
binders = chromo.binders.make_binder_collection([hp1])

In [7]:
# Confinement
confine_type = "Spherical"
confine_length = 900

In [8]:
num_beads = 393216
bead_spacing = 16.5
chem_mods_path = np.array(["chromo/chemical_mods/meth"])
chemical_mods = Chromatin.load_seqs(chem_mods_path)
states = np.zeros(chemical_mods.shape, dtype=int)

In [ ]:
# Polymer
p = Chromatin.confined_gaussian_walk(
    'Chr-1',
    num_beads,
    bead_spacing,
    states=np.zeros(chemical_mods.shape, dtype=int),
    confine_type=confine_type,
    confine_length=confine_length,
    binder_names=np.array(['HP1']),
    chemical_mods=chemical_mods,
    chemical_mod_names=np.array(['H3K9me3'])
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p.r[:,0], p.r[:,1], p.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
# Field
n_bins_x = 63
x_width = 2 * confine_length * (1 + 1/n_bins_x)
n_bins_y = n_bins_x
y_width = x_width
n_bins_z = n_bins_x
z_width = x_width
udf = UniformDensityField(
    [p], binders, x_width, n_bins_x, y_width,
    n_bins_y, z_width, n_bins_z, confine_type=confine_type,
    confine_length=confine_length, chi=1,
    assume_fully_accessible=1, fast_field=1, n_points=1000
)

### Determine Chemical Modification States at Various Levels of Coarse-Graining

In [11]:
intervals_1_100 = rd.get_cg_bead_intervals(num_beads, cg_factor=100)
intervals_1_50 = rd.get_cg_bead_intervals(num_beads, cg_factor=50)
intervals_1_25 = rd.get_cg_bead_intervals(num_beads, cg_factor=25)
intervals_1_5 = rd.get_cg_bead_intervals(num_beads, cg_factor=5)

chemical_mods_1_100 = rd.get_majority_state_in_interval(chemical_mods, intervals_1_100)
chemical_mods_1_50 = rd.get_majority_state_in_interval(chemical_mods, intervals_1_50)
chemical_mods_1_25 = rd.get_majority_state_in_interval(chemical_mods, intervals_1_25)
chemical_mods_1_5 = rd.get_majority_state_in_interval(chemical_mods, intervals_1_5)

num_beads_1_100 = len(chemical_mods_1_100)
num_beads_1_50 = len(chemical_mods_1_50)
num_beads_1_25 = len(chemical_mods_1_25)
num_beads_1_5 = len(chemical_mods_1_5)

print(f"Number of Beads in 1:100 scale simulation: {num_beads_1_100}")
print(f"Number of Beads in 1:50 scale simulation: {num_beads_1_50}")
print(f"Number of Beads in 1:25 scale simulation: {num_beads_1_25}")
print(f"Number of Beads in 1:5 scale simulation: {num_beads_1_5}")

Number of Beads in 1:100 scale simulation: 3933
Number of Beads in 1:50 scale simulation: 7865
Number of Beads in 1:25 scale simulation: 15729
Number of Beads in 1:5 scale simulation: 78644


### Determine Bead Spacing at Various Levels of Coarse-Graining

In [ ]:
bead_spacing_1_50 = np.sqrt(
    2 * p.lp**2 * (
        (50 * bead_spacing) / lp - 1 +
        np.exp(-((50 * bead_spacing) / lp))
    )
)

bead_spacing_1_25 = np.sqrt(
    2 * p.lp**2 * (
        (25 * bead_spacing) / lp - 1 +
        np.exp(-((25 * bead_spacing) / lp))
    )
)

bead_spacing_1_5 = np.sqrt(
    2 * p.lp**2 * (
        (5 * bead_spacing) / lp - 1 +
        np.exp(-((5 * bead_spacing) / lp))
    )
)

### Initialize Binders at Various Levels of Coarse-Graining

In [ ]:
binders_cg_50 = rd.get_cg_binders(
    binders_refined = binders,
    cg_factor = 50
)

binders_cg_25 = rd.get_cg_binders(
    binders_refined = binders,
    cg_factor = 25
)

binders_cg_5 = rd.get_cg_binders(
    binders_refined = binders,
    cg_factor = 5
)

### Coarse-Grain the Original Polymer

In [ ]:
cg_factor = 100

In [ ]:
p_cg_100 = rd.get_cg_chromatin(
    polymer = p,
    cg_factor = cg_factor,
    name_cg = "Chr_CG_100"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg.r[:,0], p_cg.r[:,1], p_cg.r[:,2], s=10, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
udf_cg_100 = rd.get_cg_udf(
    udf_refined_dict = udf.dict_,
    binders_refined = binders,
    cg_factor = cg_factor,
    polymers_cg = [p_cg_100]
)

In [ ]:
binders_cg_100 = rd.get_cg_binders(
    binders_refined = binders,
    cg_factor = cg_factor
)

### Equilibrate the Coarse-Grained Polymer

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_cg_100])
num_snapshots = 200
mc_steps_per_snapshot = 1000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_cg = mc.polymer_in_field(
    [p_cg_100],
    binders_cg_100,
    udf_cg_100,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_cg_100 = polymers_cg[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_100.r[:,0], p_cg_100.r[:,1], p_cg_100.r[:,2], s=10, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### First Level of Refinement

In [ ]:
n_bind_eq = 100000
p_cg_50, udf_cg_50 = rd.refine_chromatin(
    polymer_cg = p_cg_100,
    num_beads_refined = num_beads_1_50,
    bead_spacing = bead_spacing_1_50,
    chemical_mods = chemical_mods_1_50,
    udf_cg = udf_cg_100,
    binding_equilibration = n_bind_eq,
    name_refine = "Chr_CG_50",
    output_dir = "output"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_50.r[:,0], p_cg_50.r[:,1], p_cg_50.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_cg_50])
num_snapshots = 200
mc_steps_per_snapshot = 5000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp_mild"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_cg = mc.polymer_in_field(
    [p_cg_50],
    binders_cg_50,
    udf_cg_50,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_cg_50 = polymers_cg[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_50.r[:,0], p_cg_50.r[:,1], p_cg_50.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### Second Level of Refinement

In [ ]:
n_bind_eq = 100000
p_cg_25, udf_cg_25 = rd.refine_chromatin(
    polymer_cg = p_cg_50,
    num_beads_refined = num_beads_1_25,
    bead_spacing = bead_spacing_1_25,
    chemical_mods = chemical_mods_1_25,
    udf_cg = udf_cg_50,
    binding_equilibration = n_bind_eq,
    name_refine = "Chr_CG_25",
    output_dir = "output"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_25.r[:,0], p_cg_25.r[:,1], p_cg_25.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_cg_25])
num_snapshots = 200
mc_steps_per_snapshot = 5000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp_mild"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_cg = mc.polymer_in_field(
    [p_cg_25],
    binders_cg_25,
    udf_cg_25,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_cg_25 = polymers_cg[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_25.r[:,0], p_cg_25.r[:,1], p_cg_25.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### Third Level of Refinement

In [ ]:
n_bind_eq = 100000
p_cg_5, udf_cg_5 = rd.refine_chromatin(
    polymer_cg = p_cg_25,
    num_beads_refined = num_beads_1_5,
    bead_spacing = bead_spacing_1_5,
    chemical_mods = chemical_mods_1_5,
    udf_cg = udf_cg_25,
    binding_equilibration = n_bind_eq,
    name_refine = "Chr_CG_5",
    output_dir = "output"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_5.r[:,0], p_cg_5.r[:,1], p_cg_5.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_cg_5])
num_snapshots = 200
mc_steps_per_snapshot = 5000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp_mild"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_cg = mc.polymer_in_field(
    [p_cg_5],
    binders_cg_5,
    udf_cg_5,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_cg_5 = polymers_cg[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_cg_5.r[:,0], p_cg_5.r[:,1], p_cg_5.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### Refine the Coarse-Grained Polymer

In [ ]:
n_bind_eq = 1000000
p_refine, udf_refine = rd.refine_chromatin(
    polymer_cg = p_cg_5,
    num_beads_refined = num_beads,
    bead_spacing = bead_spacing,
    chemical_mods = chemical_mods,
    udf_cg = udf_cg_5,
    binding_equilibration = n_bind_eq,
    name_refine = "Chr_refine",
    output_dir = "output"
)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_refine.r[:,0], p_refine.r[:,1], p_refine.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()

### Equilibrate the Refined Polymer

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds([p_refine])
num_snapshots = 200
mc_steps_per_snapshot = 20000

# Create a list of mu schedules, which are defined in another file
schedules = [func[0] for func in getmembers(ms, isfunction)]
select_schedule = "linear_step_for_negative_cp_mild"
mu_schedules = [
    ms.Schedule(getattr(ms, func_name)) for func_name in schedules
]
mu_schedules = [sch for sch in mu_schedules if sch.name == select_schedule]

In [ ]:
polymers_refined = mc.polymer_in_field(
    [p_refine],
    binders,
    udf,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mu_schedule=mu_schedules[0],
    random_seed=np.random.randint(0, 1E5)
)

In [ ]:
p_refine = polymers_refined[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_refine.r[:,0], p_refine.r[:,1], p_refine.r[:,2], s=0.5, alpha=0.5)
ax.set_xticks(np.arange(-900, 901, 300))
ax.set_yticks(np.arange(-900, 901, 300))
ax.set_zticks(np.arange(-900, 901, 300))
plt.show()